<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/pull_one_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)





In [119]:

tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

lowerPrice_result = rets_client.search(resource = 'Property', resource_class='Property',limit = 20, dmql_query='(ModificationTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
count = 0
di ={}
df = pd.DataFrame()
for item in lowerPrice_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
      if(item['PreviousListPrice']>item['ListPrice']):
        df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
        count += 1
        di = item
    
      
